In [30]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import util

In [31]:
ksi_collapse = pd.read_csv('Data/Created Datasets/KSI_Collapse_Count_truncate3.csv', index_col = 0)
road_df = pd.read_csv('Data/Road_Impediments_Rounded3.csv')

In [32]:
pd.set_option('display.max_columns', None)

In [25]:
road_df.head()

,Latitude,Longitude,PercentOfVehicles,AvgAcceleration,PercentCar,PercentMPV,PercentLDT,PercentMDT,PercentHDT,PercentOther
0,37.801,-95.952,0.008,0.268,0.000,0.030,0.653,0.065,0.201,0.050
1,43.645,-79.393,0.094,0.732,0.023,0.261,0.298,0.094,0.179,0.145
2,43.609,-79.555,0.002,0.562,0.022,0.125,0.279,0.067,0.363,0.143
3,43.685,-79.283,0.092,0.467,0.004,0.236,0.407,0.107,0.052,0.195
4,43.669,-79.426,0.045,0.520,0.011,0.306,0.229,0.107,0.127,0.221


In [26]:
# don't need to do this with newly queried road impediments dataset

road_df['Latitude'] = road_df['Latitude'].apply(util.truncate, args = (3, ))
road_df['Longitude'] = road_df['Longitude'].apply(util.truncate, args = (3, ))

In [27]:
# DON'T NEED TO DO THIS WITH THE NEWLY QUERIED ROAD IMPEDIMENTS DATASET
# IT HAS ALREADY BEEN AGGREGATED BY LAT AND LONG IN MYSQL
# EITHER WAYS - RESULTS WILL BE SAME

road_collapse = road_df.pivot_table(values = ['AvgAcceleration', 'PercentOfVehicles', 'PercentCar', 'PercentMPV', 'PercentLDT', 'PercentMDT', 'PercentHDT', 'PercentOther'], index = ['Latitude', 'Longitude'], aggfunc = np.mean).reset_index()

In [28]:
road_collapse

,Latitude,Longitude,AvgAcceleration,PercentCar,PercentHDT,PercentLDT,PercentMDT,PercentMPV,PercentOfVehicles,PercentOther
0,37.798,-95.952,0.367,0.0,0.249,0.665,0.035,0.009,0.132,0.042
1,37.798,-95.951,0.178,0.0,0.230,0.702,0.033,0.000,0.003,0.035
2,37.798,-95.950,1.036,0.0,0.214,0.722,0.026,0.000,0.031,0.039
3,37.798,-95.949,0.219,0.0,0.180,0.743,0.038,0.000,0.017,0.038
4,37.798,-95.948,0.479,0.0,0.061,0.887,0.028,0.000,0.004,0.024
...,...,...,...,...,...,...,...,...,...,...
49995,44.573,-96.638,0.000,0.0,0.216,0.620,0.043,0.104,0.000,0.017
49996,44.573,-96.637,0.000,0.0,0.214,0.624,0.043,0.103,0.000,0.017
49997,44.573,-96.634,0.709,0.0,0.214,0.629,0.037,0.103,0.006,0.017
49998,44.573,-96.633,0.000,0.0,0.216,0.629,0.035,0.103,0.000,0.017


In [33]:
road_merge = road_collapse.merge(ksi_collapse, left_on = ['Latitude', 'Longitude'], right_on = ['LATITUDE', 'LONGITUDE'], how = 'inner')

In [34]:
road_merge.drop(['Latitude', 'Longitude'], axis = 1, inplace = True)

In [35]:
road_merge.sort_values(by = 'COUNT', ascending = False)

,AvgAcceleration,PercentCar,PercentHDT,PercentLDT,PercentMDT,PercentMPV,PercentOfVehicles,PercentOther,LATITUDE,LONGITUDE,COUNT
593,0.419,0.079,0.131,0.358,0.079,0.215,0.044,0.137,43.650,-79.386,13
760,0.646,0.064,0.211,0.321,0.069,0.162,0.039,0.173,43.656,-79.452,10
3290,0.241,0.007,0.109,0.342,0.055,0.423,0.041,0.065,43.821,-79.316,8
584,0.292,0.026,0.164,0.363,0.092,0.172,0.015,0.184,43.650,-79.439,8
2839,0.688,0.052,0.386,0.213,0.061,0.115,0.056,0.171,43.781,-79.494,8
...,...,...,...,...,...,...,...,...,...,...,...
1346,0.471,0.000,0.163,0.185,0.056,0.322,0.099,0.274,43.676,-79.505,1
1347,1.058,0.000,0.132,0.206,0.044,0.397,0.044,0.221,43.676,-79.502,1
1349,0.496,0.026,0.306,0.227,0.074,0.150,0.023,0.217,43.676,-79.494,1
1350,0.356,0.024,0.204,0.295,0.104,0.129,0.008,0.243,43.676,-79.489,1


In [36]:
road_merge.to_csv('KSI_Road_truncate3.csv')